# Roboquant

Roboquant is a framework written in Kotlin for algoritmic trading. It enables robust back-testing and live trading in a fully automatic way. As an algo-trader, you can use roboquant in two ways:

- In a Jupyter Notebook 
- As a library in your standalone application using your IDE of choice

Both share exactly the same code-base, so there is no difference how to develop your strategies or how they behave. The Notebook approach is a great to quickly be up and running and have a more iterative approach to developing new trading strategies. It comes also with charts that visualize how your strategy is poerformning. And as your strategy grows, you might benefit from the power of a full-fledged IDE like IDEA and move to a standolne application.

This notebook shows the very basics of using roboquant and how to test a simple strategy. We start with the cell magic **%use roboquant** to load the required packages. The **Welcome()** statement is optional, but gives some info on the environment we are running in.



In [ ]:
%use roboquant
Welcome()

# Define an Roboquant
An Roboquant is the engine of tyhe framework. It ties all the components toegther that make-up a back-test and also runs the actual tests. 

There several sensible default parameters for a roboquant, so the two main ingredients remaining to get started are:

1. The strategy we want to test. For this example we use a Exponential Moving Average Crossover with its default settings. This comes out-of-the-box with roboquant.
2. Any metrics we want to captured during the run to see how our strategy is performing. In this case we use the AccountSummary.

roboquant allows for much more customization, but for this introduction notebook this will be fine.  

In [ ]:
val strategy = EMACrossover()
val roboquant =  Roboquant(strategy, AccountSummary())

# Feed & Run
Now we define the data we want to use for the run. In roboqaunt data is provided by a Feed and there are many Feed providers included out of the bo. They are ranging from Feeds for local CSV files to online data providers. In this notebook we use a the Avro Feed provider which also comes with some S&P 500 test data. 

After the feed is defined, we are all set to start the run. When you start a run, by default a progress bar is shown that provides visual feedback how long the run might take. 

In [ ]:
val feed = AvroFeed.sp500()
roboquant.run(feed)

# Examine Results

Likely the first thing you want to know after the run has finished is how does the account look like. An account contains: 

- the cash balances
- the portfolio with its positions
- the orders, both open and closed orders
- any trades made, so orders that have been (partially) filled

Please note the use of the *summary()* method. Many classes in roboquant support this method and it provides a nice summary of the state on an object, like in this case the account object.

In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
// You can also get more details, for example about the portfolio
account.portfolio.summary()

In [ ]:
val trades = account.trades
trades[0..10].summary()

# Metric results
The account contains the state at the end of the run. Metrics are more powerful in that they capture state during each step of the run, so they give an overview how a metric evolves over time during the run.  

In [ ]:
var logger = roboquant.logger
logger.getMetricNames()

In [ ]:
logger.summary(2)

In [ ]:
logger.getMetric("account.value").summary()

# More detailed inspections

After a quick scan of the results, you might want to dive into some more details. For example how much profit or loss did the strategy make on Apple?

In [ ]:
val apple = trades.assets.findBySymbol("AAPL")
trades.realizedPnL(apple)

Or how much profit did we make in each year?

In [ ]:
trades.timeline.split(Period.ofYears(1)).forEach {
    val pnl = trades.realizedPnL(it)
    println("$it profit: $pnl")
}

# Charts
A great way to get this overview more easy is by using one of the many included charts. If you want to find out more about that, check out the visualization Notebook. In this notebook just an example how to quickly spot which days had the best and worst performance.

In [ ]:
CalendarChart(logger.getMetric("account.change"))